In [1]:
import sys
import os

import argparse
import time
import json
from datetime import date

import torch
#import lightning as L
#from lightning.pytorch.callbacks import ModelCheckpoint,EarlyStopping

#Transformers
import transformers
import bitsandbytes as bnb
from transformers import AutoModelForCausalLM , AutoTokenizer
from transformers import pipeline, set_seed
from transformers import get_linear_schedule_with_warmup, AdamW
from transformers import AutoConfig
from transformers import BitsAndBytesConfig
#from lightning.pytorch.loggers import TensorBoardLogger

#Dataset
from datasets import load_dataset

#PEFT
from peft import LoraConfig
from peft import PeftConfig
from peft import PeftModel
from peft import get_peft_model
from peft import prepare_model_for_kbit_training


import warnings
warnings.filterwarnings("ignore")

torch.set_float32_matmul_precision('medium')
torch.cuda.empty_cache()







2024-03-27 15:57:26.951554: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 15:57:27.015403: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 15:57:29.364095: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} 

In [2]:
# Define a function to print the number of trainable parameters in the given model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"Trainable params: {trainable_params} || All params: {all_param} || Trainable %: {100 * trainable_params / all_param}")

def tokenize_input(df,tokenizer,tokenizer_chapter_max_length,tokenizer_summary_max_length):

    prompt_start = "Summarize the following : \n"
    prompt_end = "\n\nSummary:"

    prompt = [prompt_start + dialogue + prompt_end for dialogue in df["chapter"]]

    df["input_ids"] = tokenizer(prompt, max_length=tokenizer_chapter_max_length , padding="max_length" , truncation=True , return_tensors="pt").input_ids
    df["labels"] = tokenizer(df["summary_text"],max_length=tokenizer_summary_max_length , padding="max_length" , truncation=True , return_tensors="pt").input_ids

    return df

In [3]:
cache_dir = "/work/LitArt/nair/cache/" 
log_path = "/work/LitArt/nair/outdir/"

tokenizer_chapter_max_length = 1024
tokenizer_summary_max_length = 256
model = "meta-llama/Llama-2-7b-hf"
tokenizer_name = "meta-llama/Llama-2-7b-hf"





today = date.today()


log_path = log_path+model.replace("/","-")+"-" +str(today)+"-"+time.strftime("%H:%M:%S", time.localtime())
#logger = TensorBoardLogger(log_path, name="my_model")


In [4]:
from transformers import AutoTokenizer
#Bits and Bytes config
bnb_config = BitsAndBytesConfig(
load_in_4bit=True, #4bit quantizaition - load_in_4bit is used to load models in 4-bit quantization 
bnb_4bit_use_double_quant=True, #nested quantization technique for even greater memory efficiency without sacrificing performance. This technique has proven beneficial, especially when fine-tuning large models
bnb_4bit_quant_type="nf4", #quantization type used is 4 bit Normal Float Quantization- The NF4 data type is designed for weights initialized using a normal distribution
bnb_4bit_compute_dtype=torch.bfloat16, #modify the data type used during computation. This can result in speed improvements. 
)
model = AutoModelForCausalLM.from_pretrained(model,
                                                    device_map="auto",
                                                    trust_remote_code=True,
                                                    quantization_config=bnb_config,
                                                    cache_dir=cache_dir)


tokenizer = AutoTokenizer.from_pretrained(tokenizer_name,cache_dir=cache_dir)


# Set the padding token of the tokenizer to its end-of-sentence token
tokenizer.pad_token = tokenizer.eos_token

tokenizer.add_special_tokens({'pad_token': '<PAD>'})

# Enable gradient checkpointing for the model. Gradient checkpointing is a technique used to reduce the memory consumption during the backward pas. Instead of storing all intermediate activations in the forward pass (which is what's typically done to compute gradients in the backward pass), gradient checkpointing stores only a subset of them
model.gradient_checkpointing_enable() 

# Prepare the model for k-bit training . Applies some preprocessing to the model to prepare it for training.
model = prepare_model_for_kbit_training(model)


#If only targeting attention blocks of the model
#target_modules = ["q_proj", "v_proj"]

#If targeting all linear layers
target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']

    
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules = target_modules,
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model.add_adapter(lora_config)

#base_model = get_peft_model(base_model, lora_config)

# Print the number of trainable parameters in the model
print_trainable_parameters(model)





Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Trainable params: 40554496 || All params: 3540967424 || Trainable %: 1.1452942414869276


In [5]:
data = load_dataset('csv', 
                    data_files={
                        'train': "/work/LitArt/data/chunked_dataset/train_dataset_with_summaries.csv",
                        'test': "/work/LitArt/data/chunked_dataset/test_dataset_with_summaries.csv",
                        'val':"/work/LitArt/data/chunked_dataset/validation_dataset_with_summaries.csv"})

In [6]:
data


DatasetDict({
    train: Dataset({
        features: ['chapter', 'human_summary', '__index_level_0__', 'summary_text'],
        num_rows: 10668
    })
    test: Dataset({
        features: ['chapter', 'human_summary', '__index_level_0__', 'summary_text'],
        num_rows: 1614
    })
    val: Dataset({
        features: ['chapter', 'human_summary', '__index_level_0__', 'summary_text'],
        num_rows: 1548
    })
})

In [7]:
tokenized_dataset = data["train"].shuffle().map(tokenize_input, batched=True, fn_kwargs={"tokenizer": tokenizer, "tokenizer_chapter_max_length": tokenizer_chapter_max_length,"tokenizer_summary_max_length":tokenizer_summary_max_length})
tokenized_dataset = tokenized_dataset.remove_columns(['chapter', 'human_summary', '__index_level_0__', 'summary_text'])

Map:   0%|          | 0/10668 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments

#Training Parameters
batch_size = 3
epochs= 4
output_dir = f"llama-7b-qlora-Capstone-project"
per_device_train_batch_size = batch_size
gradient_accumulation_steps = 4
optim = 'adamw_hf' #"paged_adamw_32bit" #"paged_adamw_8bit"
save_steps = 10
save_total_limit=3
logging_steps = 10
learning_rate = 1e-5
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "constant" #"cosine"
epochs=1, 

training_arguments = TrainingArguments(
    output_dir=log_path,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=epochs, 
    #save_steps=save_steps,
    save_total_limit=save_total_limit,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    save_strategy='epoch',
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    #push_to_hub=True,
)


In [9]:
from trl import SFTTrainer

def formatting_func(example):
    text = f"### USER: Summarize the following text : {example['chapter']}\n### ASSISTANT: {example['summary_text']}"
    return text



In [10]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=data["test"],
    packing=True,
    #dataset_text_field="id",
    tokenizer=tokenizer,
    max_seq_length=1024,
    formatting_func=formatting_func,
)


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
def save_hyperparameters(log_path, quantization_config, lora_config , training_arguments):
    import os
    os.makedirs(log_path, exist_ok=True)
    
    file_path = os.path.join(log_path, 'hyperparameters.txt')  
    
    with open(file_path, 'w') as file:
        file.write(str(quantization_config))
        file.write(str(lora_config))
        file.write(str(training_arguments))
        
    
save_hyperparameters(log_path, bnb_config , lora_config , training_arguments)



In [ ]:
trainer.train() # [ 410/1000 42:51 < 1:01:58, 0.16 it/s, Epoch 2.74/7]at 3pm

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.925400
20,2.877100
30,2.912400
40,2.804000
50,2.843600
60,2.819800
70,2.789400
80,2.749200
90,2.732500
100,2.738600


In [ ]:
model.save_pretrained(log_path)

In [ ]:
trainer.save_model(log_path)

In [ ]:
model_dir = log_path

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)


tokenizer = AutoTokenizer.from_pretrained(model_dir)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=quantization_config,
    #adapter_kwargs={"revision": "09487e6ffdcc75838b10b6138b6149c36183164e"}
)


In [ ]:


def generate_response(chapter : str) -> str:
    prompt =  f"""### USER: Summarize the following text : ' {chapter}' ### Assistant:  """.strip()
    inputs = tokenizer(prompt, return_tensors="pt").to(0)
    outputs = model.generate(inputs.input_ids, max_new_tokens=50, do_sample=False)
    return(tokenizer.decode(outputs[0], skip_special_tokens=False))


'''
	encoding = tokenizer(prompt, return_tensors = "pt").to(DEVICE)
	#with torch.inference_mode():
    with torch.no_grad():
		outputs = model.generate(
			input_ids=encoding.input_ids,
			attention_mask=encoding.attention_mask,
			generation_config=generation_config,
		)

	response = tokenizer.decode(outputs[0], skip_special_tokens=True)
	#assistant_start =  "<assistant>:"
	#response_start = response.find(assistant_start)
	#return response[response_start + len(assistant_start) : ].strip()

    return response.strip()

'''


chapter = '''In a village where the mountains kissed the clouds and the rivers sang to the valleys, there lived a boy named Idris. Idris had the peculiar ability to understand the language of the wind. It was a gift that had been passed down through his family for generations, but in Idris, it found its most curious student.

Each morning, Idris would climb to the highest peak, sit among the whispers of the passing breezes, and listen. The wind spoke of distant lands, of the secrets of the forest, and of the tales of the creatures that walked within it. Idris's favorite stories were those of the Guardians of the Forest, mythical beings said to protect the balance between nature and the world of men.

One day, a tempest unlike any other approached the village. The wind's voice was frantic, warning of a darkness that sought to devour the forest and everything within it. Idris understood what he had to do. He remembered the tales of the Guardians and knew that if he could find them, they could save his home.

With nothing but the clothes on his back and the courage in his heart, Idris ventured into the forest. The wind guided him, whispering paths through the twisting undergrowth and overgrown trails. After days of journeying deeper than any villager had dared, Idris found the heart of the forest. It was there, in a clearing bathed in moonlight, that he met the Guardians.

They were not what he expected. The Guardians were the forest itself — the trees, the rivers, the stones, and the wind. They spoke in a chorus of natural harmony, and Idris understood them. He pleaded for their help, telling them of the impending darkness.

The Guardians listened and then spoke in a voice like the rustling of leaves. "The darkness you speak of is born from the hearts of men," they said. "It cannot be fought with force but with understanding. Return to your people, Idris. Teach them to listen as you have listened."

Idris returned to his village as the storm broke upon the mountains. He told them of the Guardians, of the language of the wind, and of the darkness that was born from their own actions. At first, they did not understand, but Idris did not give up. He showed them how to listen, how to care for the land that had cared for them.

In time, the village changed. The people learned to live in harmony with the land, to listen to the wind, and to respect the balance of nature. The darkness receded, the forest flourished, and Idris's village prospered.

Idris grew old, but he continued to climb to the peak each morning, to listen to the wind, and to teach others to do the same. And though the story of the boy who spoke to the wind became a legend, the lesson it taught remained: to listen, to understand, and to respect the voices of the world around us.
'''
print (generate_response(chapter))




In [ ]:
import pandas as pd
pd.DataFrame(trainer.state.log_history)

In [8]:
model_dir = "meta-llama/Llama-2-7b-hf"

In [9]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)


tokenizer = AutoTokenizer.from_pretrained(model_dir)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=quantization_config,
    #adapter_kwargs={"revision": "09487e6ffdcc75838b10b6138b6149c36183164e"}
)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:


def generate_response(chapter : str) -> str:
    prompt =  f"""### USER: Summarize the following text : ' {chapter}' ### Assistant:  """.strip()
    inputs = tokenizer(prompt, return_tensors="pt").to(0)
    outputs = model.generate(inputs.input_ids, max_new_tokens=500, do_sample=False)
    return(tokenizer.decode(outputs[0], skip_special_tokens=False))


'''
	encoding = tokenizer(prompt, return_tensors = "pt").to(DEVICE)
	#with torch.inference_mode():
    with torch.no_grad():
		outputs = model.generate(
			input_ids=encoding.input_ids,
			attention_mask=encoding.attention_mask,
			generation_config=generation_config,
		)

	response = tokenizer.decode(outputs[0], skip_special_tokens=True)
	#assistant_start =  "<assistant>:"
	#response_start = response.find(assistant_start)
	#return response[response_start + len(assistant_start) : ].strip()

    return response.strip()

'''


chapter = '''In a village where the mountains kissed the clouds and the rivers sang to the valleys, there lived a boy named Idris. Idris had the peculiar ability to understand the language of the wind. It was a gift that had been passed down through his family for generations, but in Idris, it found its most curious student.

Each morning, Idris would climb to the highest peak, sit among the whispers of the passing breezes, and listen. The wind spoke of distant lands, of the secrets of the forest, and of the tales of the creatures that walked within it. Idris's favorite stories were those of the Guardians of the Forest, mythical beings said to protect the balance between nature and the world of men.

One day, a tempest unlike any other approached the village. The wind's voice was frantic, warning of a darkness that sought to devour the forest and everything within it. Idris understood what he had to do. He remembered the tales of the Guardians and knew that if he could find them, they could save his home.

With nothing but the clothes on his back and the courage in his heart, Idris ventured into the forest. The wind guided him, whispering paths through the twisting undergrowth and overgrown trails. After days of journeying deeper than any villager had dared, Idris found the heart of the forest. It was there, in a clearing bathed in moonlight, that he met the Guardians.

They were not what he expected. The Guardians were the forest itself — the trees, the rivers, the stones, and the wind. They spoke in a chorus of natural harmony, and Idris understood them. He pleaded for their help, telling them of the impending darkness.

The Guardians listened and then spoke in a voice like the rustling of leaves. "The darkness you speak of is born from the hearts of men," they said. "It cannot be fought with force but with understanding. Return to your people, Idris. Teach them to listen as you have listened."

Idris returned to his village as the storm broke upon the mountains. He told them of the Guardians, of the language of the wind, and of the darkness that was born from their own actions. At first, they did not understand, but Idris did not give up. He showed them how to listen, how to care for the land that had cared for them.

In time, the village changed. The people learned to live in harmony with the land, to listen to the wind, and to respect the balance of nature. The darkness receded, the forest flourished, and Idris's village prospered.

Idris grew old, but he continued to climb to the peak each morning, to listen to the wind, and to teach others to do the same. And though the story of the boy who spoke to the wind became a legend, the lesson it taught remained: to listen, to understand, and to respect the voices of the world around us.
'''
print (generate_response(chapter))




<s> ### USER: Summarize the following text : ' In a village where the mountains kissed the clouds and the rivers sang to the valleys, there lived a boy named Idris. Idris had the peculiar ability to understand the language of the wind. It was a gift that had been passed down through his family for generations, but in Idris, it found its most curious student.

Each morning, Idris would climb to the highest peak, sit among the whispers of the passing breezes, and listen. The wind spoke of distant lands, of the secrets of the forest, and of the tales of the creatures that walked within it. Idris's favorite stories were those of the Guardians of the Forest, mythical beings said to protect the balance between nature and the world of men.

One day, a tempest unlike any other approached the village. The wind's voice was frantic, warning of a darkness that sought to devour the forest and everything within it. Idris understood what he had to do. He remembered the tales of the Guardians and knew

In [ ]:
# Load the configuration for the trained model
config = PeftConfig.from_pretrained(model_dir)

In [19]:
# Load the trained model using the loaded configuration and other parameters
model = AutoModelForCausalLM.from_pretrained(
	config.base_model_name_or_path,
	return_dict=True,
	quantization_config=bnb_config,
	device_map="auto",
	trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
# Load the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)



In [21]:
# Set the padding token of the tokenizer to its end-of-sentence token
tokenizer.pad_token = tokenizer.eos_token

In [22]:
#Inference
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id





In [27]:
def generate_response(chapter : str) -> str:
    prompt =  f"""### USER:Summarize the following text ' {chapter}' ### Assistant:  """.strip()
    encoding = tokenizer(prompt, return_tensors = "pt").to(DEVICE)
    #with torch.inference_mode():
    with torch.no_grad():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #assistant_start =  "<assistant>:"
    #response_start = response.find(assistant_start)
    #return response[response_start + len(assistant_start) : ].strip()

    return response.strip()




In [28]:
#prompt

chapter = '''In a village where the mountains kissed the clouds and the rivers sang to the valleys, there lived a boy named Idris. Idris had the peculiar ability to understand the language of the wind. It was a gift that had been passed down through his family for generations, but in Idris, it found its most curious student.

Each morning, Idris would climb to the highest peak, sit among the whispers of the passing breezes, and listen. The wind spoke of distant lands, of the secrets of the forest, and of the tales of the creatures that walked within it. Idris's favorite stories were those of the Guardians of the Forest, mythical beings said to protect the balance between nature and the world of men.

One day, a tempest unlike any other approached the village. The wind's voice was frantic, warning of a darkness that sought to devour the forest and everything within it. Idris understood what he had to do. He remembered the tales of the Guardians and knew that if he could find them, they could save his home.

With nothing but the clothes on his back and the courage in his heart, Idris ventured into the forest. The wind guided him, whispering paths through the twisting undergrowth and overgrown trails. After days of journeying deeper than any villager had dared, Idris found the heart of the forest. It was there, in a clearing bathed in moonlight, that he met the Guardians.

They were not what he expected. The Guardians were the forest itself — the trees, the rivers, the stones, and the wind. They spoke in a chorus of natural harmony, and Idris understood them. He pleaded for their help, telling them of the impending darkness.

The Guardians listened and then spoke in a voice like the rustling of leaves. "The darkness you speak of is born from the hearts of men," they said. "It cannot be fought with force but with understanding. Return to your people, Idris. Teach them to listen as you have listened."

Idris returned to his village as the storm broke upon the mountains. He told them of the Guardians, of the language of the wind, and of the darkness that was born from their own actions. At first, they did not understand, but Idris did not give up. He showed them how to listen, how to care for the land that had cared for them.

In time, the village changed. The people learned to live in harmony with the land, to listen to the wind, and to respect the balance of nature. The darkness receded, the forest flourished, and Idris's village prospered.

Idris grew old, but he continued to climb to the peak each morning, to listen to the wind, and to teach others to do the same. And though the story of the boy who spoke to the wind became a legend, the lesson it taught remained: to listen, to understand, and to respect the voices of the world around us.
'''

print (generate_response(chapter))





### USER:' In a village where the mountains kissed the clouds and the rivers sang to the valleys, there lived a boy named Idris. Idris had the peculiar ability to understand the language of the wind. It was a gift that had been passed down through his family for generations, but in Idris, it found its most curious student.

Each morning, Idris would climb to the highest peak, sit among the whispers of the passing breezes, and listen. The wind spoke of distant lands, of the secrets of the forest, and of the tales of the creatures that walked within it. Idris's favorite stories were those of the Guardians of the Forest, mythical beings said to protect the balance between nature and the world of men.

One day, a tempest unlike any other approached the village. The wind's voice was frantic, warning of a darkness that sought to devour the forest and everything within it. Idris understood what he had to do. He remembered the tales of the Guardians and knew that if he could find them, they co